In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime as dt
from datetime import timedelta
from sklearn import metrics
from statistics import mode
from itertools import combinations
from scipy.spatial import distance
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics import calinski_harabaz_score

#import plotly
#import plotly.plotly as py
#import plotly.graph_objs as go

In [3]:
#username = 'rafaelbraga'
#api_key = '7j8DwV4I45XDMqSpYgnC'
#plotly.tools.set_credentials_file(username=username, api_key=api_key)
#plotly.offline.init_notebook_mode(connected=True)

In [4]:
#Passo 1 - Normalizar os dados.
#Passo 2 - Para cada valor de K = 2, ..., Kmax, fazer
#        2.1 Aplicar o algoritmo K-médias por Nr rodadas.
#        2.2 Escolher os protótipos da rodada que produzir menor SSD.
#        2.3 Calcular o valor do índice de validação escolhido.
#Passo 3 - Escolher o valor ótimo Kopt como aquele que otimiza o índice de validação de agrupamentos escolhido.
#Passo 4 - Particionar os dados entre os Kopt agrupamentos usando o critério da distância euclidiana ao protótipo mais próximo.

def best_K_v3(data, validation_index):
    n_inter = 30
    K = range(2,25)
    
    plots = []
    scores = []
    best_Ks = []
    fig = plt.figure()

    inertias = np.zeros((24, 30), dtype=float)
    for k in K:
        ch, inertias = k_means_with_validation(data, k, n_inter, inertias)
        scores.append(ch)
        if ((k == 2) or (k == 5) or (k == 10) or (k == 15) or (k == 20) or (k == 24)):
            p = go.Scatter(y=inertias.mean(axis=1), x=len(inertias[k-2]), name="k = %d" % (k), 
                           error_y=dict(visible=True, arrayminus=inertias.std(axis=1)), line=dict(width=1))
            plots.append(p)

    plot_inertias(plots)
    best_Ks.append(scores.index(max(scores))+2)
    try:
        print (best_Ks)
        return mode(best_Ks), scores
    except Exception:
        print("Função do índice de validação encontrou mais de um número ideal de clusters.")
        sys.exit()

def plot_inertias(plots):
    layout = go.Layout(title = "Evolution of SSD values over the iterations.", 
                       xaxis = dict(title='Iterations', range=[0, 30]), yaxis = dict(title='SSD'), legend=dict(x=0.9, y=1))
    fig = go.Figure(data=plots, layout=layout)
    plotly.offline.plot(fig)
    py.image.save_as(fig, filename='evolution_of_SSD.png')
    #py.iplot(fig)    

def k_means_with_validation(data, k, n_inter, inertias):
    inertia_min = 0
    inertias_l = np.zeros(n_inter)
    for i in range(n_inter):
        model = MiniBatchKMeans(n_clusters=k)
        inertia = model.fit(data).inertia_
        if inertia_min == 0 or inertia < inertia_min:
            pred = model.fit_predict(data)
            labels = model.labels_
            inertia_min = inertia
            inertias_l[i] = inertia_min
    ch_index = calinski_harabaz_score(data, labels)
    inertias[k-2] = inertias_l
    return ch_index, inertias

In [5]:
#Passo 1 - Normalizar os dados.
#Passo 2 - Para cada valor de K = 2, ..., Kmax, fazer
#        2.1 Aplicar o algoritmo K-médias por Nr rodadas.
#        2.2 Escolher os protótipos da rodada que produzir menor SSD.
#        2.3 Calcular o valor do índice de validação escolhido.
#Passo 3 - Escolher o valor ótimo Kopt como aquele que otimiza o índice de validação de agrupamentos escolhido.
#Passo 4 - Particionar os dados entre os Kopt agrupamentos usando o critério da distância euclidiana ao protótipo mais próximo.

def best_K_v2(data, validation_index):

    n_inter = 30
    K = range(2,25)
    
    #plots = []
    scores = []
    best_Ks = []
    #fig = plt.figure()

    inertias = np.zeros((24, 30), dtype=float)
    for k in K:
        ch, inertias = k_means_with_validation(data, k, n_inter, inertias)
        scores.append(ch)
        #if ((k == 2) or (k == 5) or (k == 10) or (k == 15) or (k == 20) or (k == 24)):
            #p = go.Scatter(y=inertias.mean(axis=1), x=len(inertias[k-2]), name="k = %d" % (k), 
                           #error_y=dict(visible=True, arrayminus=inertias.std(axis=1)), line=dict(width=1))
            #plots.append(p)

    #layout = go.Layout(title = "Evolution of SSD values over the iterations.", 
                       #xaxis = dict(title='Iterations', range=[0, 30]), yaxis = dict(title='SSD'), legend=dict(x=0.9, y=1))
    #fig = go.Figure(data=plots, layout=layout)
    #plotly.offline.plot(fig)
    #py.image.save_as(fig, filename='evolution_of_SSD.png')
    #py.iplot(fig)
    
    best_Ks.append(scores.index(max(scores))+2)
    try:
        print (best_Ks)
        
        return mode(best_Ks), scores
    except Exception:
        print("Função do índice de validação encontrou mais de um número ideal de clusters.")
        sys.exit()
        
def k_means_with_validation(data, k, n_inter, inertias):
    inertia_min = 0
    inertias_l = np.zeros(n_inter)
    for i in range(n_inter):
        model = MiniBatchKMeans(n_clusters=k)
        inertia = model.fit(data).inertia_
        if inertia_min == 0 or inertia < inertia_min:
            pred = model.fit_predict(data)
            labels = model.labels_
            inertia_min = inertia
            inertias_l[i] = inertia_min
    ch_index = calinski_harabaz_score(data, labels)
    inertias[k-2] = inertias_l
    return ch_index, inertias

In [6]:
def best_K(data, validation_index):
    repeat = 10 
    K = range(2,16)
    all_scores = np.zeros(shape=(repeat,len(K)))
    best_Ks = []
    for r in range(repeat):
        scores = []
        for k in K:
            model = MiniBatchKMeans(n_clusters=k).fit(data)
            if(validation_index == 'Calinski-Harabasz'):
                scores.append(calinski_harabaz_score(data, model.labels_))
            elif(validation_index == 'Silhouette'):
                scores.append(metrics.silhouette_score(data, model.labels_, metric='euclidean'))
        best_Ks.append(scores.index(max(scores))+2)
        all_scores[r] = scores
    try:
        return mode(best_Ks), np.sum(all_scores,axis=0)/repeat
    except Exception:
        print("Função do índice de validação encontrou mais de um número ideal de clusters.")
        sys.exit()

def k_means(data, K):
    inertia_min = 0
    for _ in range(10):
        model = MiniBatchKMeans(n_clusters=K)
        inertia = model.fit(data).inertia_
        if inertia_min == 0 or inertia < inertia_min:
            pred = model.fit_predict(data)
            labels = model.labels_
            inertia_min = inertia
    return labels, round(inertia_min,2)

def k_means_individual(data):
    kmeans = MiniBatchKMeans(n_clusters=1)
    pred = kmeans.fit_predict(data)
    return np.round(kmeans.cluster_centers_,2)

In [7]:
def treat_columns(data_original, cols_to_celsius, cols_to_clean, cols_to_dismiss, cols_to_kilo, cols_to_standardize):
    ##########CONVERTENDO PARA CELSIUS E KILO##########
    for i in range(len(cols_to_celsius)):
        data_original[cols_to_celsius[i]] = data_original[cols_to_celsius[i]].apply(to_celsius)
    for i in range(len(cols_to_celsius)):
        data_original[cols_to_kilo[i]] = data_original[cols_to_kilo[i]].apply(to_kilo)
    ##########REMOVENDO OUTLIERS##########
    low_fences = []
    up_fences = []
    for i in range(len(cols_to_clean)):
        low_fences.append(tukey_fences(data_original[cols_to_clean[i]])[0])
        up_fences.append(tukey_fences(data_original[cols_to_clean[i]])[1])
    for i in range(len(cols_to_clean)):
        data_original = data_original[data_original[cols_to_clean[i]] > low_fences[i]]
        data_original = data_original[data_original[cols_to_clean[i]] < up_fences[i]]
    
    data_cluster = data_original.copy()
    data_norm = data_original.copy()
    ##########DELETANDO COLUNAS##########
    try:
        for i in range(len(cols_to_dismiss)):
            del data_cluster[cols_to_dismiss[i]]
            del data_norm[cols_to_dismiss[i]]
    except KeyError:
        print("Problema nas colunas para dispensar.")
        sys.exit()
    ##########NORMALIZANDO##########
    try:
        for i in range(len(cols_to_standardize)):
            data_norm[cols_to_standardize[i]] = standardize(data_norm[cols_to_standardize[i]])
    except KeyError:
        print("Problema nas colunas para normalizar.")
        sys.exit()
        
    return data_original, data_cluster, data_norm

In [8]:
def treat_columns(data_original, cols_to_celsius, cols_to_clean, cols_to_dismiss, cols_to_kilo, cols_to_standardize):
    ##########CONVERTENDO PARA CELSIUS E KILO##########
    for i in range(len(cols_to_celsius)):
        data_original[cols_to_celsius[i]] = data_original[cols_to_celsius[i]].apply(to_celsius)
    for i in range(len(cols_to_celsius)):
        data_original[cols_to_kilo[i]] = data_original[cols_to_kilo[i]].apply(to_kilo)
    ##########REMOVENDO OUTLIERS##########
    low_fences = []
    up_fences = []
    for i in range(len(cols_to_clean)):
        low_fences.append(tukey_fences(data_original[cols_to_clean[i]])[0])
        up_fences.append(tukey_fences(data_original[cols_to_clean[i]])[1])
    for i in range(len(cols_to_clean)):
        data_original = data_original[data_original[cols_to_clean[i]] > low_fences[i]]
        data_original = data_original[data_original[cols_to_clean[i]] < up_fences[i]]
    
    data_cluster = data_original.copy()
    data_norm = data_original.copy()
    ##########DELETANDO COLUNAS##########
    try:
        for i in range(len(cols_to_dismiss)):
            del data_cluster[cols_to_dismiss[i]]
            del data_norm[cols_to_dismiss[i]]
    except KeyError:
        print("Problema nas colunas para dispensar.")
        sys.exit()
    ##########NORMALIZANDO##########
    try:
        for i in range(len(cols_to_standardize)):
            data_norm[cols_to_standardize[i]] = standardize(data_norm[cols_to_standardize[i]])
    except KeyError:
        print("Problema nas colunas para normalizar.")
        sys.exit()
        
    return data_original, data_cluster, data_norm

In [9]:
def tukey_fences(x):
    Q1 = x.describe()['25%']
    Q3 = x.describe()['75%']
    IQR = Q3 - Q1
    tukey_low = Q1 - 1.5*IQR
    tukey_up = Q3 + 1.5*IQR
    return tukey_low, tukey_up

In [10]:
###########  Utils  ###########
def date_to_num(timestamp):
    return mdates.date2num(timestamp)        

def read_file(file):
    try:
        dataframe = pd.read_csv(file)
    except FileNotFoundError:
        print("Arquivo de dados não encontrado.")
        sys.exit()
    return dataframe

def standardize(x):
    return (x - x.mean())/x.std(ddof=0)

def to_celsius(x):
    return ((x-32)*(5/9))

def to_kilo(x):
    return (x*0.453592)

def to_dtdate(timestamp):
    #if len(timestamp) == 10:
        #return (dt.strptime(timestamp.split(' ')[0], '%Y-%m-%d'))
        return (dt.strptime(timestamp, '%d-%m-%Y'))
    #else:
        #return (dt.strptime(timestamp, '%Y-%m-%d %H:%M:%S'))

In [20]:
###################################EDITÁVEIS#####################################
Consider_date = True
Execution_is_for_real = True
Print_in_screen = False
validation_index = 'Calinski-Harabasz' #OPÇÕES: 'Calinski-Harabasz' OU 'Silhouette'
beehive = 'HT101_original'
#beehive = '../emil/emil'
date_col = 'Date'
begin_date = '12-9-2017'
interval = 7 #INTERVALO TOTAL A SER ANALISADO (EM DIAS)
window = 7 #JANELA A SER ANALISADA A CADA ITERAÇÃO (EM DIAS)
shift = 7 #DESLOCAMENTO DA JANELA (EM DIAS)
#PARA NÃO HAVER INTERSEÇÃO, 'window' E 'shift' DEVEM SER IGUAIS
cols_to_celsius = []
cols_to_clean = ['Temp-Btm(F)','Temp-Brood(F)','Temp-Hive(F)']
cols_to_clean = ['Temp-Btm(F)','Temp-Brood(F)','Temp-Hive(F)','BRH(%)','HRH(%)','Weight(lbs)']
cols_to_dismiss = ['D1','D2','D3','Week','Month','Year']
cols_to_kilo = ['Weight(lbs)']
cols_to_standardize = ['Temp-Btm(F)','Temp-Brood(F)','Temp-Hive(F)','BRH(%)','HRH(%)','Weight(lbs)' ]
#################################################################################
data_file = beehive+'.csv'
begin_date = to_dtdate(begin_date)
interval, shift = timedelta(days=interval), timedelta(days=shift)
window = np.timedelta64(window, 'D')
final_date = begin_date+interval

In [21]:
if Print_in_screen:
    print("Report: {} to {}".format(dt.strftime(begin_date,'%d-%m-%Y'), dt.strftime(final_date,'%d-%m-%Y')))
    print("Window: {}; Shift: {}\n".format(str(window), str(shift)))
if Execution_is_for_real:
    
    txt_out = beehive+'_'+dt.strftime(begin_date,'%d-%m-%Y')+'_'+dt.strftime(final_date,'%d-%m-%Y')+'_clusters.txt'
    brief = beehive+'_'+dt.strftime(begin_date,'%d-%m-%Y')+'_'+dt.strftime(final_date,'%d-%m-%Y')+'_brief.txt'
    
    with open(txt_out, 'w') as f:
        f.write("Report: "+dt.strftime(begin_date,'%d-%m-%Y')+" to "+dt.strftime(final_date,'%d-%m-%Y')+"\n")
        f.write("Window: "+str(window)+"; Shift: "+str(shift)+"\n\n")
    with open(brief, 'w') as f:
        f.write("Report: "+dt.strftime(begin_date,'%d-%m-%Y')+" to "+dt.strftime(final_date,'%d-%m-%Y')+"\n")
        f.write("Window: "+str(window)+"; Shift: "+str(shift)+"\n\n")

score_out = beehive+'_'+dt.strftime(begin_date,'%d-%m-%Y')+'_'+dt.strftime(final_date,'%d-%m-%Y')+'_scores.txt'

with open(score_out, 'w') as f:
    f.write("Report: "+dt.strftime(begin_date,'%d-%m-%Y')+" to "+dt.strftime(final_date,'%d-%m-%Y')+"\n")
    f.write("Window: "+str(window)+"; Shift: "+str(shift)+"\n\n")        

#LENDO ARQUIVO E TRANSFORMANDO A COLUNA DA DATA PARA OBJETO DATETIME
dataframe_original = read_file(data_file)
date = dataframe_original[date_col].apply(to_dtdate)

#ITERANDO AS DATAS DE ACORDO COM 'WINDOW' E 'SHIFT'
todos_centroids = []
for i in np.arange(begin_date, final_date, shift):
    dataframe = dataframe_original.copy()
    if (i+window > final_date):
        interval_end = final_date
    else:
        interval_end = i+window
    dataframe = dataframe[(date >= i) & (date <= interval_end)]
    i, interval_end = pd.to_datetime(str(i)).strftime('%d-%m-%Y'), pd.to_datetime(str(interval_end)).strftime('%d-%m-%Y')
    if Print_in_screen:
        print("--------------------------------------------------------------------------------")
        print("Interval: {} to {}".format(i, interval_end))
    if Execution_is_for_real:
            csv_out = beehive+'_'+i+'_'+interval_end+'_raw.csv'
            csv_norm_out = beehive+'_'+i+'_'+interval_end+'_norm.csv'
            with open(txt_out, 'a') as f:
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Interval: "+i+" to "+interval_end+"\n")
            with open(brief, 'a') as f:
                f.write("--------------------------------------------------------------------------------\n")
                f.write("Interval: "+i+" to "+interval_end+"\n")

    dataframe, dataframe_cluster, dataframe_norm = treat_columns(dataframe, cols_to_celsius, cols_to_clean, cols_to_dismiss, cols_to_kilo, cols_to_standardize)
    
    if Consider_date:
        date_num = dataframe[date_col].apply(to_dtdate).apply(date_to_num)
        dataframe_cluster[date_col], dataframe_norm[date_col] = date_num, standardize(date_num)

    if((validation_index != 'Calinski-Harabasz') and (validation_index != 'Silhouette')):
        print("Deve ser escolhido um índice de validação válido: Calinski-Harabasz ou Silhouette.")
        sys.exit()
    else:
        #K, all_scores = best_K_v2(dataframe_norm, validation_index)
        K = 1   
        with open(score_out, 'a') as f:
            f.write("--------------------------------------------------------------------------------\n")
            f.write("Interval: "+i+" to "+interval_end+"\n")
            #f.write(str(all_scores)+"\n")
        #print(all_scores)
        labels, ssd = k_means(dataframe_norm, K)
        dataframe['KMeansLabel'] = labels
        if Print_in_screen:
            print("Ideal K by {} index: {}".format(validation_index, K))
            print("Sum of squared distances: {}\n".format(ssd))
        if Execution_is_for_real:
            dataframe.to_csv(csv_out, index=False)
            dataframe_norm.to_csv(csv_norm_out, index=False)
            with open(txt_out, 'a') as f:
                f.write("Ideal K by "+str(validation_index)+" index: "+str(K)+"\n")
                f.write("Sum of squared distances: "+str(ssd)+"\n\n")
            with open(brief, 'a') as f:
                f.write("Ideal K by "+str(validation_index)+" index: "+str(K)+"\n")
                f.write("Sum of squared distances: "+str(ssd)+"\n\n")

    centroids = []
    for k in range(K):
        centroid = k_means_individual(dataframe_cluster[labels==k])
        centroids.append(centroid)
        todos_centroids.append(centroid)
        descriptive_statistics = round(dataframe_cluster[labels==k].describe(),2)
        if Print_in_screen:
            print("Cluster {}\nCentroid: {}\n".format(k, centroid))
            print("Descriptive Statistics:\n", descriptive_statistics, "\n")
        if Execution_is_for_real:
            with open(txt_out, 'a') as f:
                f.write("Cluster "+str(k)+"\nCentroid: "+str(centroid)+"\n\n")
                f.write("Descriptive Statistics:\n"+str(descriptive_statistics)+"\n\n")
  
   

    for ((i,xy1),(j,xy2)) in combinations(enumerate(centroids), 2):
        distance_between_centroids = round(distance.euclidean(xy1, xy2),2)
        if Print_in_screen:
            print("Distance between centroids {} and {}: {}".format(i,j,distance_between_centroids))
        if Execution_is_for_real:
            with open(txt_out, 'a') as f:
                f.write("Distance between centroids "+str(i)+" and "+str(j)+": "+str(distance_between_centroids)+"\n")
            with open(brief, 'a') as f:
                f.write("Distance between centroids "+str(i)+" and "+str(j)+": "+str(distance_between_centroids)+"\n")
print (todos_centroids)

Arquivo de dados não encontrado.


SystemExit: 

C:\Users\Lia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
def dist_centroids_different_timestamps(todos_centroids):
    
    if Execution_is_for_real:
        centroid_relation = beehive+'_'+dt.strftime(begin_date,'%d-%m-%Y')+'_'+dt.strftime(final_date,'%d-%m-%Y')+'_centroid_relation.txt'
        with open(centroid_relation, 'a') as f:
            
            f.write("Date clusteres {} to {}".format(dt.strftime(begin_date,'%d-%m-%Y'), dt.strftime(final_date,'%d-%m-%Y'))+"\n\n")
   
    for ((i,xy1),(j,xy2)) in combinations(enumerate(todos_centroids), 2):
        distance_between_centroids = round(distance.euclidean(xy1, xy2),2)
        if Execution_is_for_real:
            centroid_relation = beehive+'_'+dt.strftime(begin_date,'%d-%m-%Y')+'_'+dt.strftime(final_date,'%d-%m-%Y')+'_centroid_relation.txt'
            with open(centroid_relation, 'a') as f:
                f.write("Distance between centroids c"+str(i)+" and c"+str(j)+": "+str(distance_between_centroids)+"\n")
        print("Distance between centroids {} and {}: {}".format(i,j,distance_between_centroids))
       

In [14]:

dist_centroids_different_timestamps(todos_centroids)

Distance between centroids 0 and 1: 20.11
Distance between centroids 0 and 2: 22.15
Distance between centroids 0 and 3: 34.08
Distance between centroids 0 and 4: 34.54
Distance between centroids 0 and 5: 47.57
Distance between centroids 0 and 6: 57.55
Distance between centroids 0 and 7: 62.87
Distance between centroids 0 and 8: 68.52
Distance between centroids 0 and 9: 74.17
Distance between centroids 0 and 10: 86.07
Distance between centroids 0 and 11: 89.25
Distance between centroids 0 and 12: 93.82
Distance between centroids 0 and 13: 101.52
Distance between centroids 0 and 14: 111.19
Distance between centroids 0 and 15: 116.4
Distance between centroids 0 and 16: 120.71
Distance between centroids 0 and 17: 127.81
Distance between centroids 0 and 18: 135.9
Distance between centroids 0 and 19: 142.69
Distance between centroids 0 and 20: 146.8
Distance between centroids 0 and 21: 154.74
Distance between centroids 0 and 22: 159.63
Distance between centroids 0 and 23: 167.99
Distance bet

Distance between centroids 10 and 11: 14.61
Distance between centroids 10 and 12: 21.15
Distance between centroids 10 and 13: 25.2
Distance between centroids 10 and 14: 38.0
Distance between centroids 10 and 15: 44.87
Distance between centroids 10 and 16: 49.43
Distance between centroids 10 and 17: 55.97
Distance between centroids 10 and 18: 60.3
Distance between centroids 10 and 19: 68.37
Distance between centroids 10 and 20: 73.08
Distance between centroids 10 and 21: 80.08
Distance between centroids 10 and 22: 86.79
Distance between centroids 10 and 23: 94.37
Distance between centroids 10 and 24: 101.28
Distance between centroids 10 and 25: 109.33
Distance between centroids 11 and 12: 9.12
Distance between centroids 11 and 13: 16.37
Distance between centroids 11 and 14: 39.84
Distance between centroids 11 and 15: 45.04
Distance between centroids 11 and 16: 47.79
Distance between centroids 11 and 17: 53.77
Distance between centroids 11 and 18: 58.33
Distance between centroids 11 and 

ModuleNotFoundError: No module named 'Orange'

SyntaxError: invalid syntax (<ipython-input-3-d24c364f0d6d>, line 4)

In [ ]:
print(km)